In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
332048,3.102002e+09,3,310,2,31020021,12,1,C,1.0,1.0,1,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
174825,1.332002e+09,1,133,2,13320019,10,2,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166927,1.321201e+09,1,132,1,13212006,2,1,C,1.0,1.0,1,12.0,1.0,19.0,1.0,1.0,NaN,NaN,NaN
130664,1.241100e+09,1,124,1,12411002,3,6,C,NaN,NaN,1,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
46771,1.092005e+09,1,109,2,10920053,10,2,C,2.0,1.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [3]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [4]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [5]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
449394,4.092001e+09,balochistan,kalat,rural,40920006,B,6,female,present,son/daughter,2005,3,0,2,unmarried
343524,3.052002e+09,nwfp,shangla,rural,30520023,B,2,female,present,wife/husband,1953,0,0,53,married
81698,1.142001e+09,punjab,sialkot,rural,11420010,B,5,female,present,son/daughter,1996,0,0,10,unmarried
213480,2.022000e+09,sindh,sukkur,rural,20220001,B,4,female,present,son/daughter,1993,0,0,13,unmarried
294061,2.152000e+09,sindh,thaeparkar,rural,21520002,B,1,male,present,head of household,1971,0,0,35,married


In [6]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [7]:
# Fix NAs
C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].fillna(value=str('no'))
C_relevant['currently_enrolled'].isna().value_counts()

False    446425
Name: currently_enrolled, dtype: int64

In [8]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [9]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [10]:
# Select people in school enrollment age (I would say 4-15 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 15)]

In [11]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    66049
2    42560
3    34792
4    29827
Name: province, dtype: int64

In [12]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")
years06_07.loc[(years06_07["region"] == 1), 'region'] = 'urban'
years06_07.loc[(years06_07["region"] == 2), 'region'] = 'rural'

In [13]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
75,NWFP,Karak,31220
60,Sindh,Sanghar,21320
17,Punjab,Chakwal,10420
82,NWFP,Kohistan,31920
37,Punjab,Multan,12420


In [14]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
20,Punjab,Khushab,10720,1072
69,NWFP,Malakand,30620,3062
62,Sindh,Tharparkar,21520,2152
48,Sindh,Khairpur,20120,2012
56,Sindh,Dadu,20920,2092


In [15]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          144401
left_only          0
right_only         0
Name: _merge, dtype: int64

In [16]:
years06_07_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
87046,2161207308.0,11,6,male,unmarried,1,0.0,1.0,NaN,urban,2,2161,Sindh,Karachi,21611,both
64684,2072000508.0,3,12,male,unmarried,2,NaN,no,14.0,rural,2,2072,Sindh,Shikarpur,20720,both
51309,1332001811.0,5,14,female,unmarried,1,6.0,1.0,NaN,rural,1,1332,Punjab,Bahawalnager,13320,both


In [17]:
# add year
years06_07_['year']=2005
years06_07_['marital_status']=years06_07_['marital_status'].astype('str')

#fix marital status categories
years06_07_.loc[(years06_07_['marital_status'] == "unmarried"), 'marital_status'] = 1
years06_07_.loc[(years06_07_['marital_status'] == "married"), 'marital_status'] = 2
years06_07_.loc[(years06_07_['marital_status'] == "only nikha has been solemanised,rukhsati not taken"), 'marital_status'] = 5
years06_07_.loc[(years06_07_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years06_07_.loc[(years06_07_['marital_status'] == "divorced"), 'marital_status'] = 4
years06_07_['marital_status']=years06_07_['marital_status'].astype('int')

# fix column names
years06_07_['province']=years06_07_['province_y']

In [18]:
#select relevant columns
years06_07_ = years06_07_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "subprovince",
        "year",
    ]
].copy()

In [19]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.csv', index_label="id")

# PART 2

In [20]:
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("yes", 1)
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("no", 0)
years06_07_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince,year
64098,2062003706.0,5,10,1,female,2,0.0,rural,Sindh,2062,Jaccobabad,2005
82665,2152002910.0,4,15,1,male,1,1.0,rural,Sindh,2152,Tharparkar,2005
17722,1121101009.0,4,13,1,male,2,0.0,urban,Punjab,1121,Gujranwala,2005
120750,4011201404.0,8,15,1,male,2,0.0,urban,Balochistan,4011,Quetta,2005
134863,4162001413.0,4,6,1,male,2,0.0,rural,Balochistan,4162,Gwadar,2005


In [21]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer

# check region for anomalies

years06_07_["currently_enrolled"]= years06_07_["currently_enrolled"].astype('int').copy()
df_grp_1 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()
df_grp_2 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()

# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          408
left_only       0
right_only      0
Name: _merge, dtype: int64

In [22]:
df_grp_merge.isna().any()

sex                     False
subprovince             False
region                  False
currently_enrolled_x    False
currently_enrolled_y    False
_merge                  False
dtype: bool

In [23]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]

# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})

# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
144,male,Okara,rural,739,535,0.723951
325,female,Mardan,urban,0,0,NaN
64,male,Jafarabad,rural,538,299,0.555762
199,male,Vehari,urban,0,0,NaN
283,female,Kasur,urban,0,0,NaN


In [24]:
df_grp_merge.to_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years06_07_merge_diff.csv")

In [25]:
df_grp_merge.isna().any()

sex                  False
subprovince          False
region               False
sample_population    False
enrolled_total       False
rate_enrollment       True
dtype: bool